In [1]:
import pandas as pd
#import seaborn as sns
#import matplotlib.pyplot as plt
#import numpy as np
import sqlite3 as sql
import re

# Task 1

In [ ]:
# TODO

# Task 2

In [2]:
conn = sql.connect("user_actions.db")
cursor = conn.cursor()

# check what tables we have
list(cursor.execute("SELECT name FROM sqlite_master WHERE type='table';"))

[('user_actions',)]

In [3]:
# check columns for the table user_actions
list(cursor.execute("SELECT * FROM PRAGMA_TABLE_INFO('user_actions');"))

[(0, 'user_id', 'INTEGER', 0, None, 0),
 (1, 'username', 'TEXT', 0, None, 0),
 (2, 'email', 'TEXT', 0, None, 0),
 (3, 'action', 'TEXT', 0, None, 0),
 (4, 'timestamp', 'TEXT', 0, None, 0)]

In [4]:
# how many rows do we have in the table user_actions?
list(cursor.execute("SELECT count(*) FROM user_actions;"))

[(8040,)]

In [5]:
# Retrieve the usernames of all users who have performed the "signup" action
query = """
SELECT
    username 
FROM
    user_actions
WHERE
    action='signup'
ORDER BY
    username;
"""

list(cursor.execute(query))

[('user1',),
 ('user10',),
 ('user11',),
 ('user12',),
 ('user13',),
 ('user14',),
 ('user15',),
 ('user16',),
 ('user17',),
 ('user18',),
 ('user19',),
 ('user2',),
 ('user20',),
 ('user21',),
 ('user22',),
 ('user23',),
 ('user24',),
 ('user25',),
 ('user26',),
 ('user27',),
 ('user28',),
 ('user29',),
 ('user3',),
 ('user30',),
 ('user31',),
 ('user32',),
 ('user33',),
 ('user34',),
 ('user35',),
 ('user4',),
 ('user5',),
 ('user6',),
 ('user7',),
 ('user8',),
 ('user9',)]

In [6]:
# Find the total number of log entries for each user. Display the user_id, username, and the count of log entries.
query = """
SELECT
    user_id,
    username,
    COUNT(*) 
FROM
    user_actions
GROUP BY
    user_id
ORDER BY
    user_id;
"""

list(cursor.execute(query))

[(1, 'user1', 104),
 (2, 'user2', 149),
 (3, 'user3', 108),
 (4, 'user4', 436),
 (5, 'user5', 192),
 (6, 'user6', 457),
 (7, 'user7', 362),
 (8, 'user8', 329),
 (9, 'user9', 118),
 (10, 'user10', 170),
 (11, 'user11', 328),
 (12, 'user12', 209),
 (13, 'user13', 470),
 (14, 'user14', 320),
 (15, 'user15', 35),
 (16, 'user16', 379),
 (17, 'user17', 367),
 (18, 'user18', 122),
 (19, 'user19', 58),
 (20, 'user20', 362),
 (21, 'user21', 323),
 (22, 'user22', 275),
 (23, 'user23', 309),
 (24, 'user24', 162),
 (25, 'user25', 136),
 (26, 'user26', 217),
 (27, 'user27', 211),
 (28, 'user28', 91),
 (29, 'user29', 49),
 (30, 'user30', 165),
 (31, 'user31', 389),
 (32, 'user32', 32),
 (33, 'user33', 32),
 (34, 'user34', 180),
 (35, 'user35', 394)]

In [7]:
# Identify users who have both logged in (action = 'login') and signed up (action = 'signup') on the same day.
# Display the user_id and username.

query = """
SELECT
    ua1.user_id,
    ua1.username
FROM
    user_actions ua1
JOIN
    user_actions ua2 ON
    ua1.user_id = ua2.user_id AND 
    DATE(ua1.timestamp) = DATE(ua2.timestamp) AND 
    ua1.action = 'login' AND
    ua2.action = 'signup'
ORDER BY
    ua1.user_id
"""

list(cursor.execute(query))

[(2, 'user2'),
 (5, 'user5'),
 (5, 'user5'),
 (7, 'user7'),
 (8, 'user8'),
 (12, 'user12'),
 (22, 'user22'),
 (22, 'user22'),
 (30, 'user30'),
 (32, 'user32')]

# Task 3

In [8]:
with open("comments.txt", 'r') as file:
    comments = file.readlines()

# extract all hashtags from a given comment
regexp_all_hashtags=r"#\w+"

tags_in_comment_1 = re.findall(regexp_all_hashtags, comments[0])
print("Tags in comment 1: " + ', '.join(tags_in_comment_1))

#  find comments that mention both "#programming" and "#python"
regexp_programming_and_python = r"(?=.*#programming)(?=.*#python)"

matching_comment_2 = bool(re.search(regexp_programming_and_python, comments[1]))
print("\nDoes comment 2 contain both #programming and #python?\n" + str(matching_comment_2))

# Bonus task:
# Extract unique hashtags from the entire file by:
# - using the regexp from above
# - add all hashtags to a set so duplicates are removed
# - joining the list of comments to a single string 
unique_hashtags = set(re.findall(regexp_all_hashtags, ''.join(comments)))
print("\nUnique hashtags:")
print(unique_hashtags)


Tags in comment 1: #programming, #tips

Does comment 2 contain both #programming and #python?
False

Unique hashtags:
{'#tips', '#python', '#insights', '#tech', '#analysis', '#research', '#data', '#programming', '#coding', '#analytics', '#innovation'}
